# Sprint 論文読解入門

## 論文
[[8]Ren, S., He, K., Girshick, R., Sun, J.: Faster r-cnn: Towards real-time object detection with region proposal networks. In: Advances in neural information processing systems. (2015) 91–99](https://arxiv.org/pdf/1506.01497.pdf)

### 参考
- [Faster R-CNNにおけるRPNの世界一分かりやすい解説](https://medium.com/lsc-psd/faster-r-cnn%E3%81%AB%E3%81%8A%E3%81%91%E3%82%8Brpn%E3%81%AE%E4%B8%96%E7%95%8C%E4%B8%80%E5%88%86%E3%81%8B%E3%82%8A%E3%82%84%E3%81%99%E3%81%84%E8%A7%A3%E8%AA%AC-dfc0c293cb69)
- [Faster R-CNN (object detection) implemented by Keras for custom data from Google’s Open Images Dataset V4](https://towardsdatascience.com/faster-r-cnn-object-detection-implemented-by-keras-for-custom-data-from-googles-open-images-125f62b9141a)
- [Fast-CNN](https://www.cv-foundation.org/openaccess/content_iccv_2015/papers/Girshick_Fast_R-CNN_ICCV_2015_paper.pdf)

## 問題
それぞれについてJupyter Notebookにマークダウン形式で記述してください。


(1) 物体検出の分野にはどういった手法が存在したか。


(2) Fasterとあるが、どういった仕組みで高速化したのか。


(3) One-Stageの手法とTwo-Stageの手法はどう違うのか。


(4) RPNとは何か。


(5) RoIプーリングとは何か。


(6) Anchorのサイズはどうするのが適切か。


(7) 何というデータセットを使い、先行研究に比べどういった指標値が得られているか。


(8) （アドバンス課題）Faster R-CNNよりも新しい物体検出の論文では、Faster R-CNNがどう引用されているか。


-----条件-----
1. 答える際は論文のどの部分からそれが分かるかを書く。
2. 必要に応じて先行研究（引用されている論文）も探しにいく。最低2つは他の論文を利用して回答すること。
3. 論文の紹介記事を見ても良い。ただし、答えは論文内に根拠を探すこと。

### (1)物体検出の分野にはどういった手法が存在したか。

[モデル]
- SPPnet
- Fast R-CNN

[手法]
- region proposal methods
- region-based convolutional neural networks (RCNNs) 
- Selective Search
- EdgeBoxes

### (2) Fasterとあるが、どういった仕組みで高速化したのか。
- 従来のR-CNNは切り取った矩形領域を一つ一つNNにかけていたが、CNNと組み合わせ重みを共有することで矩形切取→物体検出までを一気通貫でできるようにした。
>- (P2.The first module is a deep
fully convolutional network that proposes regions,
and the second module is the Fast R-CNN detector [2]
that uses the proposed regions. The entire system is a single, unified network for object detection (Figure 2).)
> - (P3. Because our ultimate goal is to share computation with a Fast R-CNN
object detection network )


### (3) One-Stageの手法とTwo-Stageの手法はどう違うのか。
- Two-stage概要
  - 回帰と分類それぞれの計算ができる構造  
  (P10.The OverFeat paper [9] proposes a detection
method that uses regressors and classifiers on sliding
windows over convolutional feature maps.)
  - 特徴
  領域ごとの特徴が適応的になり、領域の特徴を具体的にカバーできる。  
    (P10.In the second stage of
our cascade, the region-wise features are adaptively
pooled [1], [2] from proposal boxes that more faithfully cover the features of the regions. We believe
these features lead to more accurate detections.)

  
- 関連論文
  1. [R-CNN minus R](https://arxiv.org/pdf/1506.06981.pdf)
    - Two-stageについて記述箇所。要約すると「領域をCNNに取り込み総合的に判断する」とあり。総合的というのは回帰・分類ということではないかと推測。  
    (P2.  The two-stage computation using region
proposal generation is a form of cascade detector [20]
or jumping window [17, 19]. However, they differ in
part-based detector such as [7] in that they do not explicitly model object parts in learning; instead parts
are implicitly capture in the CNN. Integrated training of SPP-CNN as a single CNN learning problem,
not dissimilar to some of the ideas of Section 3.2,
have very recently been explored in the unpublished
manuscript [8].)

### (4) RPNとは何か。
- Region Proposal Network
  - 役割：矩形領域を複数取得し、それぞれにスコアをつける。(P3.A Region Proposal Network (RPN) takes an image
(of any size) as input and outputs a set of rectangular
object proposals, each with an objectness score.3)　　

  - スコア（重み）をつけ、もう1層（stage）のCNNとスコア共有をする。Fast R-CNNモジュールがどこを見るべきかを指示(P3.the RPN　module tells the Fast R-CNN module where to look.)

### (5) RoIプーリングとは何か。
- 畳み込み処理が終わった「feature map」からresion proporsalを取得したのちに、全てのregion proposalを元の画像サイズにプーリングさせる手法。これにより共通の次元サイズでNN計算ができる。バックプロバゲーション時にはプーリング層は無視される。ただし解像度が落ちてしまうという欠点あり。
>- (P5.To account
for varying sizes, a set of k bounding-box regressors
are learned. Each regressor is responsible for one scale
and one aspect ratio, and the k regressors do not share
weights. As such, it is still possible to predict boxes of
various sizes even though the features are of a fixed
size/scale, thanks to the design of anchors.
>- (P5.Next we describe algorithms that
learn a unified network composed of RPN and Fast
R-CNN with shared convolutional layers)
>- (P6.These
gradients are ignored in the above approximate joint
training)


### (6) Anchorのサイズはどうするのが適切か。
- Anchorとは
  - Feature mapからピクセル等間隔で割り振っていく中心点のようなもの
  
- AnchorBoxとは
  - 各Anchorから基準の長さとアスペクト比を設定して作る矩形。

- R-CNNでは基準の長さを(128^2, 256^2, 512^2)、アスペクト比を（1:1, 1:2, 2:1（正方形、横長、縦長））としていて、1つのAnchorに対し基準の長さとアスペクト比のそれぞれを組み合わせた９通りのAnchorBoxが作られるということになる。

### (7) 何というデータセットを使い、先行研究に比べどういった指標値が得られているか。
- データセット:PASCAL VOC 2007 , PASCAL VOC 2012, MS COCO


- 先行研究との比較指標値
  >- mAP(mean Average Precision):全ての時点iにおけるAP(平均適合率)を平均することで、より一般的な平均値を求める為のもの。
  において、従来の手法や従来のregion proposal数（２０００）の15%の数のregion proposal数(300)で59.9%という指標値が得られている。
  (P7.Table2, RPN
with Fast R-CNN achieves competitive results, with
an mAP of 59.9% while using up to 300 proposals8.)
  >- 実行時間においても、従来のSSから比べ約1/30に削減。
  (P8. Table5)
  

### (8) （アドバンス課題）Faster R-CNNよりも新しい物体検出の論文では、Faster R-CNNがどう引用されているか。

- [YOLO](https://arxiv.org/pdf/1506.02640.pdf)
  - one-stage手法
  - 損失関数を工夫（かなり複雑で一切わかりませんでしたが）
  - Fast R-CNNとYOLOを組み合わせて、mAPが70％程度に上がっている。

## その他
- [Keras Faster-RCNNのgithub](https://github.com/you359/Keras-FasterRCNN)
- 画像処理系のKerasチュートリアルを実行し、別ファイルにて保存